In [1]:
import os
import datetime
import json
import time
# web scrape
import requests
import bs4

In [ ]:
class Orchestrator:
    """
    """
    def __init__(self,name,item_names,price_range,cities,distance):
        self.name = name
        self.item_names = item_names
        self.price_range = price_range
        self.distance = distance
        
        self.base_url = 'https://{CITY}.craigslist.org/search/sss?hasPic=1&sort=rel&min_price={MIN_PRICE}&max_price={MAX_PRICE}&query={ITEM}&search_distance={DISTANCE}'

        self.cities = []
    
    def get_cities(self):
        None
    
    def get_listings(self):
        for city in self.cities:
            city.get_listings()

            for listing in city.listings:


In [ ]:
class Listing:
    """
    """
    def __init__(self,city,item,parent_url,info):
        self.city = city
        self.item = item
        self.parent_url = parent_url
        self.info = info

        self.attributes = {}
    
    def scrape(self):
        None

In [ ]:
class City:
    """
    """
    def __init__(self,item,city,urls):
        self.item = item
        self.city = city
        self.urls = urls
        self.listings = []
    
    def get_listings(self):
        """
        """
        for url in self.urls:
            response = requests.get(url)
            soup = bs4.BeautifulSoup(response.text, 'html.parser')
            listings = soup.find_all('li', class_='result-row')
            for listing in listings:
                pkg = {'city':self.city,'item':self.item,'parent_url':url,'info':listing}
                self.listings.append(Listing(**pkg))

In [2]:
class Orchestrator:
    """
    """
    
    def __init__(self,name,item_names,price_range,cities,distance):
        self.name = name
        self.item_names = item_names
        self.price_range = price_range
        self.cities = cities
        self.distance = distance
        
        self.base_url = 'https://{CITY}.craigslist.org/search/sss?hasPic=1&sort=rel&min_price={MIN_PRICE}&max_price={MAX_PRICE}&query={ITEM}&search_distance={DISTANCE}'
        
        self.urls = {
            city: [
                self.search_url.format(
                    CITY = city,
                    MIN_PRICE = self.price_range[0],
                    MAX_PRICE = self.price_range[1],
                    ITEM = item_name.replace(' ','%20'),
                    DISTANCE = self.distance,
                ) for item_name in self.item_names
            ] for city in self.cities
        }
        
        self.available = []
    
    def run(self):
        links = {}
        for city,urls in self.urls.items():
            links[city] = []
            for url in urls:
                r = requests.get(url)
                soup = bs4.BeautifulSoup(r.text,'html.parser')
                for row in soup.find_all(name='li', attrs={'class':'result-row'}):
                    links[city].append(row.find(name='a',attrs={'class':'result-title hdrlnk'})['href'])
        self.available = links

In [4]:
hobie.urls

{'washingtondc': ['https://washingtondc.craigslist.org/search/sss?hasPic=1&sort=rel&min_price=2000&max_price=5000&query=hobie%20cat&search_distance=50',
  'https://washingtondc.craigslist.org/search/sss?hasPic=1&sort=rel&min_price=2000&max_price=5000&query=hobie%20cat%20wave%2016&search_distance=50'],
 'richmond': ['https://richmond.craigslist.org/search/sss?hasPic=1&sort=rel&min_price=2000&max_price=5000&query=hobie%20cat&search_distance=50',
  'https://richmond.craigslist.org/search/sss?hasPic=1&sort=rel&min_price=2000&max_price=5000&query=hobie%20cat%20wave%2016&search_distance=50'],
 'norfolk': ['https://norfolk.craigslist.org/search/sss?hasPic=1&sort=rel&min_price=2000&max_price=5000&query=hobie%20cat&search_distance=50',
  'https://norfolk.craigslist.org/search/sss?hasPic=1&sort=rel&min_price=2000&max_price=5000&query=hobie%20cat%20wave%2016&search_distance=50'],
 'fredericksburg': ['https://fredericksburg.craigslist.org/search/sss?hasPic=1&sort=rel&min_price=2000&max_price=5000&

In [3]:
hobie_params = {
    'name': 'hobie cat',
    'price_range': (2000,5000),
    'cities': ['washingtondc','richmond','norfolk','fredericksburg','boston',],
    'distance': '50',
    'item_names': ['hobie cat','hobie cat wave 16']
}
hobie = Finder(**hobie_params)
hobie.run()
hobie.available

{'washingtondc': [],
 'richmond': [],
 'norfolk': [],
 'fredericksburg': [],
 'boston': []}